## 1. Linear Regression

In [1]:
import numpy as np

In [2]:
class CustomLinearRegression:
  def __init__(self, X_data, y_target, learning_rate=0.01, num_epochs=10000):
    self.num_samples = X_data.shape[0]
    self.X_data = np.c_[np.ones((self.num_samples, 1)), X_data]
    self.y_target = y_target
    self.learning_rate = learning_rate
    self.num_epochs = num_epochs

    # Initial weights
    self.theta = np.random.randn(self.X_data.shape[1], 1)
    self.losses = []

  def compute_loss(self, y_pred, y_target):
    loss = np.mean((y_pred - y_target) ** 2)
    return loss

  def predict(self, X_data):
    y_pred = X_data.dot(self.theta)
    return y_pred

  def fit(self):
    for epoch in range(self.num_epochs):
      y_pred = self.predict(self.X_data)
      loss = self.compute_loss(y_pred, self.y_target)
      self.losses.append(loss)

      loss_grd = 2*(y_pred - self.y_target).dot(self.X_data) / self.num_samples
      gradients = self.X_data.T.dot(loss_grd)

      self.theta -= self.learning_rate * gradients

      if (epoch % 50) == 0:
        print(f"Epoch {epoch}, Loss: {loss}")

    return {
        'loss': sum(self.losses) / len(self.losses),
        'weight': self.theta
    }

In [3]:
def r2score(y_pred, y):
  rss = np.sum((y_pred - y) ** 2)
  tss = np.sum((y - np.mean(y)) ** 2)
  r2 = 1 - (rss / tss)
  return r2

In [4]:
# # Case 1
y_pred = np.array([1, 2, 3, 4, 5])
y = np.array([1, 2, 3, 4, 5])
print(r2score(y_pred, y))

# Case 2
y_pred = np.array([1, 2, 3, 4, 5])
y = np.array([3, 5, 5, 2, 4])
print(r2score(y_pred, y))

1.0
-2.235294117647059


## 2. Polynomial Regression

In [5]:
def create_polynomial_features(X, degree=2):
  """ Creates the polynomial features
  Args:
    X: A array tensor for the data.
    degree: A intege for the degree of
    the generated polynomial function.
  """
  X_new = X
  for d in range(2, degree + 1):
    X_new = np.c_[X_new, np.power(X, d)]
  return X_new

X = np.array([[1], [2], [3]])

In [6]:
def create_polynomial_features(X, degree=2):
  """ Creates the polynomial features
  Args:
    X: A array for the data.
    degree: A intege for the degree of
    the generated polynomial function.
  """
  X_mem = []
  for X_sub in X.T:
    X_new = X_sub
    for d in range(2, degree + 1):
      X_new = np.c_[X_new, np.power(X_sub, d)]
    X_mem.extend(X_new.T)
  return np.array(X_mem).T

X = np.array ([[1, 2],
[2, 3],
[3, 4]])
degree = 2
create_polynomial_features(X, degree)

array([[ 1,  1,  2,  4],
       [ 2,  4,  3,  9],
       [ 3,  9,  4, 16]])

## 3. Sales Prediction

In [7]:
import pandas as pd
import numpy as np

### a. Load dataset

In [8]:
!gdown 1A8kK0IEsT3w8htzU18ihFr5UV-euhquC

Downloading...
From: https://drive.google.com/uc?id=1A8kK0IEsT3w8htzU18ihFr5UV-euhquC
To: /content/SalesPrediction.csv
100% 206k/206k [00:00<00:00, 69.0MB/s]


### b. Preprocessing

In [13]:
df = pd.read_csv('SalesPrediction.csv')
df = pd.get_dummies(df)
df.head()

,TV,Radio,Social Media,Sales,Influencer_Macro,Influencer_Mega,Influencer_Micro,Influencer_Nano
0,16.0,6.566231,2.907983,54.732757,False,True,False,False
1,13.0,9.237765,2.409567,46.677897,False,True,False,False
2,41.0,15.886446,2.913410,150.177829,False,True,False,False
3,83.0,30.020028,6.922304,298.246340,False,True,False,False
4,15.0,8.437408,1.405998,56.594181,False,False,True,False


In [10]:
# Handle Null values
df = df.fillna(df.mean())

# Get features
X = df[['TV', 'Radio', 'Social Media',
        'Influencer_Macro', 'Influencer_Mega', 'Influencer_Micro',
        'Influencer_Nano']]
y = df[['Sales']]

# Train Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    random_state=0)

### c. Feature Scaling

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_processed = scaler.fit_transform(X_train)
X_test_processed = scaler.transform(X_test)
scaler.mean_[0]

54.173577723283785

### d. Polynomial Features

In [16]:
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=2)

X_train_poly = poly_features.fit_transform(X_train_processed)
X_test_poly = poly_features.transform(X_test_processed)

### e. Training & Evaluation

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)

y_pred = poly_model.predict(X_test_poly)
r2_score(y_test, y_pred)

0.9951796158547627